In [34]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Thermodynamic-related objects, stores, computes thermodynamic equations and 
feeds results to higher-level solvers.
"""
###################### Change this line before release! ###############
import constants as cst
#######################################################################
class Latt_vib:
    """
    Obtain thermodynamic properties of the input frequency and temperature, 
    either as numbers or as lists. The following properties are summed over
    input frequencies:

    * Zero point energy
    * Vibrational contributions to internal energy
    * Entropy
    * Constant volume specific heat
    * Vibrational free energy
    
    All the numbers with units are normalized to the solver unit systems
    
    Inputs:
        frequency (float or nFreq*1 array)
            Phonon angular frequency
        temperature (float or nTempt*1 array)
            Temperature range to be computed
        unit (string, see constants.py)
            Unit system used in solvers
    Outputs:
        self.ezp (nTempt*1 array)
            Zero-point energy
        self.eint_vib (nTempt*1 array)
            Vibrational contributions to internal energy
        self.s (nTempt*1 array)
            Entropy
        self.cv (nTempt*1 array)
            Constant volume specific heat
        self.f_vib (nTempt*1 array)
            Vibrational free energy
    """
    def __init__(self, frequency, temperature, unit='HARTREE'):
        """
        Initialization
        """
        import numpy as np
        
        if unit != 'HARTREE':
            global cst
            cst.redefine(unit=unit)
        
        if isinstance(frequency, (int, float)):
            frequency = [frequency, ]
        
        if isinstance(temperature, (int, float)):
            temperature = [temperature, ]
        
        self._freq = np.array(frequency, dtype=float)
        self._tempt = np.array(temperature, dtype=float)
        
        if min(self._tempt) <= -1e-5:
            err_msg = 'The minimum temperature: ' + str(min(self._tempt)) + ' is < 0 K.'
            raise ValueError(err_msg)
            
        self._ezp = np.zeros(len(temperature))
        self._eint_vib = np.zeros(len(temperature))
        self._s = np.zeros(len(temperature))
        self._cv = np.zeros(len(temperature))
        self._f_vib = np.zeros(len(temperature))
    
    def _get_ezp(self):
        """
        Return to zero-point energy
        """
        import numpy as np

        ezp = np.sum(0.5 * self._freq * cst.hbar)

        return ezp
        
    def _get_eint_vib(self, temperature):
        """
        Return to vibrational contributions of internal energy
        """
        import numpy as np

        if abs(temperature) < 1e-5:
            eint_vib = 0.5 * self._freq * cst.hbar
            return eint_vib
    
        hbar_freq = self._freq * cst.hbar
        kb_t = temperature * cst.kb
        expon = np.exp(hbar_freq / kb_t)

        eint_vib = np.sum(0.5 * hbar_freq + hbar_freq / (expon - 1))

        return eint_vib

    def _get_s(self, temperature):
        """
        Return to phonon entropy
        """
        import numpy as np
    
        if abs(temperature) < 1e-5:
            s = 0
            return s
    
        hbar_freq = self._freq * cst.hbar
        kb_t = temperature * cst.kb
        expon = np.exp(hbar_freq / kb_t)
        s = np.sum(
            cst.kb * (hbar_freq / kb_t / (expon - 1) - np.log(1 - 1 / expon))
        )

        return s
    
    def _get_cv(self, temperature):
        """
        Return to constant volume specific heat
        """
        import numpy as np
    
        if abs(temperature) < 1e-5:
            cv = 0
            return cv
    
        hbar_freq = self._freq * cst.hbar
        kb_t = temperature * cst.kb
        expon = np.exp(hbar_freq / kb_t)
        cv = np.sum(hbar_freq**2 / kb_t / temperature * expon / (expon - 1)**2)
    
        return cv
    
    def _get_f_vib(self, temperature):
        """
        Return to vibrational free energy
        """
        return self._get_ezp() + self._get_eint_vib(temperature) - \
            self._get_s(temperature) * temperature
    
    @property
    def ezp(self):
        self._ezp[:] = self._get_ezp()
        
        return self._ezp
    
    @property
    def eint_vib(self):
        for idx_t, t in enumerate(self._tempt):
            self._eint_vib[idx_t] = self._get_eint_vib(t)
        
        return self._eint_vib
    
    @property
    def s(self):
        for idx_t, t in enumerate(self._tempt):
            self._s[idx_t] = self._get_s(t)
        
        return self._s
    
    @property
    def cv(self):
        for idx_t, t in enumerate(self._tempt):
            self._cv[idx_t] = self._get_cv(t)
        
        return self._cv
            
    @property
    def f_vib(self):
        for idx_t, t in enumerate(self._tempt):
            self._f_vib[idx_t] = self._get_f_vib(t)
        
        return self._f_vib
    
class EoS_fit:
    """
    Manage and fit equations of states. 
    """
    def __init__(input_data, method='BirchMurnaghan', order=3, unit='HARTREE'):
        """
        Inputs:
            input_data (Turple-like, nCalc*2 array / list)
                Input data in a list of (energy, volume) turple-like elements
            method (string)
                Name of equation of states
            order (int)
                Useful only when method = 'Polynomial'
            unit (string)
                Unit system used in solvers
        """
        import numpy as np
        import traceback

        if unit != 'HARTREE':
            global cst
            cst.redefine(unit=unit)
        
        if len(input_data) < 4:
            err_msg = 'Insufficient inputs: ' + str(len(input_data)) + ' . At least 4 inputs needed.'
            raise ValueError(err_msg)
            
        if order < 3:
            error_msg = 'Specified order of polynomial: ' + str(order) + ' is too small. At least 3-order polynomial is used.'
            raise ValueError(err_msg)
        
        self._energy = np.array(input_data, dtype=float)[:, 0]
        self._volume = np.array(input_data, dtype=float)[:, 1]

        eos_list = {
            'Birch'            : 'self._birch()',
            'BirchMurnaghan'   : 'self._birchmurnaghan()',
            'Murnaghan'        : 'self._murnaghan()',
            'PourierTarantola' : 'self._pouriertarantola()',
            'Vinet'            : 'self._vinet()',
            'Polynomial'       : 'self._polynomial(order=order)',
        }
        
        param = {
            'self'  : self,
            'order' : order
        }

        try:
            exec(eos_list[method], param)
        except KeyError:
            print('EoS method specified: ', method, ' is not supported.')
            traceback.print_exec()
    
    def get_energy(self, volume):
        """
        Get energy from fitted expression
        """
        return self._eos(volume)
    
    def get_pressure(self, volume):
        """
        Get pressure (1st deritive) from fitted expression
        """
        from sympy import diff, symbols
        
        v = symbols('v')
        pressure = diff(self._eos(v), v).evalf(subs={'v' : volume})
        
        return pressure
        
    def get_bulk_moldulus(self, volume):
        """
        Get bulk modulus (2nd deritive) from fitted expression
        """
        from sympy import diff, symbols
        
        v = symbols('v')
        bulk_moldulus = diff(self._eos(v), v, 2).evalf(subs={'v' : volume})
        
        return bulk_modulus
    
    def _birch(self):
        """
        Birch equation of states
        """
        from pymatgen.analysis.eos import EOS
        
        self._eos = EOS('birch').fit(self._volume, self._energy)
        self._e0 = self._eos._e0
        self._v0 = self._eos._v0
        self._b0 = self._eos._b0
        self._b1 = self._eos._b1
        
        return self
    
    def _birchmurnaghan(self):
        """
        Birch-Murnaghan equation of states
        """
        from pymatgen.analysis.eos import EOS
        
        self._eos = EOS('birch_murnaghan').fit(self._volume, self._energy)
        self._e0 = self._eos._e0
        self._v0 = self._eos._v0
        self._b0 = self._eos._b0
        self._b1 = self._eos._b1
        
        return self
    
    def _murnaghan(self):
        """
        Murnaghan equation of states
        """
        from pymatgen.analysis.eos import EOS
        
        self._eos = EOS('murnaghan').fit(self._volume, self._energy)
        self._e0 = self._eos._e0
        self._v0 = self._eos._v0
        self._b0 = self._eos._b0
        self._b1 = self._eos._b1
        
        return self
    
    def _pouriertarantola(self):
        """
        Pourier-Tarantola equation of states
        """
        from pymatgen.analysis.eos import EOS
        
        self._eos = EOS('pourier_tarantola').fit(self._volume, self._energy)
        self._e0 = self._eos._e0
        self._v0 = self._eos._v0
        self._b0 = self._eos._b0
        self._b1 = self._eos._b1
        
        return self
    
    def _vinet(self):
        """
        Vinet equation of states
        """
        from pymatgen.analysis.eos import EOS
        
        self._eos = EOS('vinet').fit(self._volume, self._energy)
        self._e0 = self._eos._e0
        self._v0 = self._eos._v0
        self._b0 = self._eos._b0
        self._b1 = self._eos._b1
        
        return self
    
    def _polynomial(self, order):
        """
        Equation of states as at least 3-order polynomial function
        """
        from pymatgen.analysis.eos import EOS
        
        self._eos = EOS('numerical_eos').fit(
            self._volume, self._energy, max_poly_order_factor=order
        )
        self._e0 = self._eos._e0
        self._v0 = self._eos._v0
        self._b0 = self._eos._b0
        self._b1 = self._eos._b1
        
        return self
    
    @property
    def e0(self):
        return self._e0
    
    @property
    def v0(self):
        return self._v0
    
    @property
    def b0(self):
        return self._b0
    
    @property
    def b1(self):
        return self._b1


In [43]:
import constants as cst
import numpy as np
from pymatgen.analysis.eos import EOS
from sympy import diff, symbols

energy = [-4.263437170367E+03, -4.263437833105E+03, -4.263419954286E+03, -4.263443949744E+03]
volume = [256.867612, 272.918658, 281.188635, 264.812216]

volume = np.array(volume) / cst.ang3
energy = np.array(energy) / cst.ha
eq = EOS('birch_murnaghan').fit(volume, energy)

print(eq.v0 * cst.ang3)
print(eq.e0)
print(eq.b0)
print(eq.b1)

v = symbols('v')
diff(eq(v), v).evalf(subs={'v' : 285 / cst.ang3}) * cst.mpa

264.89587644575744
-4263.44395044368
0.00784362914118776
4.023553602518618


14572.7457917050